In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
import torch.nn
import torch.optim
import torch.utils.data
import h5py
from os.path import exists
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score


# Importing LearningCutsUtils
from LearningCutsUtils import OneToOneLinear, EfficiencyScanNetwork
from LearningCutsUtils import loss_fn, effic_loss_fn, lossvars
import LearningCutsUtils.Utils as LCU

In [ ]:
x_sig_data=None
y_sig_data=None

x_bkg_data=None
y_bkg_data=None

num_sig_events=0
num_bkg_events=0

allbranches=(
    'lep1Flavor','lep1Charge','lep1Pt','lep1Eta','lep1Phi','lep1MT_Met','lep1Signal','mt_lep1','lep1_DPhiMet',
    'lep2Flavor','lep2Charge','lep2Pt','lep2Eta','lep2Phi','lep2MT_Met','lep2Signal','mt_lep2','lep2_DPhiMet',
    'Rll','Ptll',
    'nJet30','nBJet30',
    'jet0Pt','jet0Eta','jet0Phi','jet0Btagged',
    'jet1Pt','jet1Eta','jet1Phi','jet1Btagged',
    'jet2Pt','jet2Eta','jet2Phi','jet2Btagged',
    'jet3Pt','jet3Eta','jet3Phi','jet3Btagged',
    'jet4Pt','jet4Eta','jet4Phi','jet4Btagged',
    'met_Et','met_Phi',
    'METOverHT','METOverHTLep',
    'minDPhiAllJetsMet',
    'MTauTau',
    'mt2leplsp_100')

branches=(
    'lep1MT_Met',
    'lep2MT_Met',
    'met_Et',
    'Rll') 


# open signal
mass=200
split=30
filepath='/data/mhance/SUSY/Compressed/'
filebase='SusySkimSlep_v0.2_SlepSignals__'
filename='MGPy8EG_A14N23LO_SlepSlep_dir_2L2MET75_%dp0_%dp0_NoSys' % (mass,mass-split)
filesuff='.hf5'
fullname=filepath+filebase+filename+filesuff
with h5py.File(fullname,'r') as hdf5file:
    print(fullname)
    data=hdf5file[list(hdf5file.keys())[0]]
    num_sig_events=len(data["nJet30"])
    x_sig_data=data[branches]
    y_sig_data=np.ones(num_sig_events)
        
print("Extracted %7d signal events" % num_sig_events)

fullname=filepath+"SusySkimSlep_v0.2_diboson2L__diboson2L_NoSys"+filesuff
num_events=0
with h5py.File(fullname,'r') as hdf5file:
    print(fullname)
    data=hdf5file[list(hdf5file.keys())[0]]
    num_bkg_events=len(data["nJet30"])
    x_bkg_data=data[branches]
    y_bkg_data=np.zeros(num_bkg_events)
        
print("Extracted %7d background events" % num_bkg_events)

In [ ]:
x_data=None
y_data=None
if num_bkg_events>num_sig_events:
    x_data = np.concatenate((x_sig_data,x_bkg_data[:num_sig_events]))
    y_data = np.concatenate((y_sig_data,y_bkg_data[:num_sig_events]))
else:
    x_data = np.concatenate((x_sig_data[:num_bkg_events],x_bkg_data))
    y_data = np.concatenate((y_sig_data[:num_bkg_events],y_bkg_data))

print(y_sig_data)
print(y_data)

In [ ]:
# we read in the data as fields with a custom format, which is useful for keeping track of what's what, but 
# ML libraries wants everything as tuples of floats.  
#x_data=[tuple(float(i) if np.isfinite(float(i)) else 0 for i in j) for j in x_data]
x_data=[tuple(float(i) for i in j) for j in x_data]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size=int(0.1*len(x_data)), 
                                                    random_state=123)

In [ ]:
x_train_before_scaling={}
for b in branches:
    x_train_before_scaling[b]=[event[branches.index(b)] for event in x_train]

In [ ]:
fig = plt.figure(figsize=(20,30))
fig.tight_layout()
for b in range(len(branches)):
    ax=fig.add_subplot(10,5,1+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    ax.hist(x_train_before_scaling[branches[b]])
    ax.set_xlabel(branches[b])
    ax.set_ylabel("Events/Bin")

In [ ]:
N=20000 # number of points
m=4 # dimensions

In [ ]:
# now scale based on the training data:
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_train_after_scaling={}
for b in branches:
    x_train_after_scaling[b]=[event[branches.index(b)] for event in x_train]

In [ ]:
fig = plt.figure(figsize=(20,30))
fig.tight_layout()
for b in range(len(branches)):
    ax=fig.add_subplot(10,5,1+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    ax.hist(x_train_after_scaling[branches[b]])
    ax.set_xlabel(branches[b])
    ax.set_ylabel("Events/Bin")

In [ ]:
for b in branches:
    print(x_bkg_data[b])

In [ ]:
fig = plt.figure(figsize=(20,5))
fig.tight_layout()
nbins=50

for b in range(len(branches)):
    fig = plt.figure(figsize=(20,5))
    ax=fig.add_subplot(2,5,1+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    plt.yscale('log')
    ax.hist(x_sig_data[branches[b]],nbins,density=True,histtype='stepfilled',alpha=0.5,color='red')
    ax.hist(x_bkg_data[branches[b]],nbins,density=True,histtype='stepfilled',alpha=0.5,color='blue')
    ax.set_xlabel(f"Feature {branches[b]}")
    ax.set_ylabel("Events/Bin")


In [ ]:
torch.manual_seed(123)
net = torch.nn.Sequential(
    torch.nn.Linear(len(branches), 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20, 1)
)
torch.save(net.state_dict(), 'net.pth')
#loss_fn = torch.nn.MSELoss()
loss_fn = torch.nn.BCEWithLogitsLoss()
#optimizer = torch.optim.SGD(net.parameters(), lr=0.05, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

In [ ]:
x_train_tensor=torch.tensor(x_train,dtype=torch.float)
y_train_tensor=torch.tensor(y_train,dtype=torch.float)
y_train_tensor=y_train_tensor.unsqueeze(1)

print(x_train_tensor.shape)

x_test_tensor=torch.tensor(x_test,dtype=torch.float)
y_test_tensor=torch.tensor(y_test,dtype=torch.float)
y_test_tensor=y_test_tensor.unsqueeze(1)

In [ ]:
import importlib
import LearningCutsUtils.LearningCutsUtils
## needed if we change LearningCutsUtils and want to avoid reloading the kernel to see the effects
importlib.reload(LearningCutsUtils.LearningCutsUtils)
import LearningCutsUtils.LearningCutsUtils as LCU
from LearningCutsUtils import loss_fn
from LearningCutsUtils import effic_loss_fn

In [ ]:
N=20000 # number of points
m=4 # dimensions

In [ ]:
gt=1.
lt=-1.

cuts_gt_lt = [lt, gt, lt, gt]

In [ ]:
# some hyperparameters
activation_input_scale_factor=8.
learning_rate=0.02
batch_size=int(len(y_train)/20.) #
epochs=50
alpha=10.   # scaling factor to tune how important hitting the target signal efficiency is
beta=10.    # scaling factor to tune how important background rejection is
gamma=1e-3  # scaling factor for how aggressively to push the cuts to zero
target_efficiency = 0.8

In [ ]:
net = OneToOneLinear(m,activation_input_scale_factor,cuts_gt_lt)
torch.save(net.state_dict(), 'net_learningbiases.pth')
#optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
{n: theta.shape for n, theta in net.named_parameters()}

In [ ]:
losses = []
losses_test = []

net.load_state_dict(torch.load('net_learningbiases.pth',weights_only=True))

xy_train = torch.utils.data.TensorDataset(x_train_tensor.float(),y_train_tensor)
loader = torch.utils.data.DataLoader(xy_train, batch_size=batch_size, shuffle=True)


In [ ]:

debug=False

for epoch in range(epochs):
    net.train()
    start_time = time.time()
    for x_batch, y_batch in loader:
        y_pred = net(x_batch)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch.squeeze(1), m, net, target_efficiency, alpha, beta, gamma)
        loss.totalloss().backward()
        optimizer.step()
    losses.append(loss)
    net.eval() # configure the model for evaluation (testing)
    y_pred = net(x_test_tensor)
    test_loss =loss_fn(y_pred, y_test_tensor.squeeze(1), m, net, target_efficiency, alpha, beta, gamma)
    losses_test.append(test_loss)
    end_time=time.time()
    elapsed_time = end_time - start_time
    bias=net.bias[0]
    weight=net.weight[0]
    #weight={weight:4.1e}, bias={bias:4.1e}, 
    print(f"Completed epoch {epoch:2d} in {elapsed_time:4.1f}s, Train loss={loss.totalloss().data:4.1e}, Test loss={test_loss.totalloss().data:4.1e}, cut={-bias/weight:4.1e}, sig_eff={100*test_loss.signaleffic:4.1f}%, bkg_eff={100*test_loss.backgreffic:6.3f}%")

In [ ]:
LearningCutsUtils.LearningCutsUtils.plotlosses(losses,losses_test)

In [ ]:
net.eval() # configure the model for evaluation (testing)
y_pred_test = net(x_test_tensor).detach().cpu()
y_pred_train= net(x_train_tensor).detach().cpu()

LearningCutsUtils.LearningCutsUtils.make_ROC_curve(y_test, y_pred_test)